# Lens FAQ
This document answers some of the most common functionality questions you may have.

**Find the code**

Download this notebook [from github.](https://raw.githubusercontent.com/credo-ai/credoai_lens/develop/docs/notebooks/lens_faq.ipynb).

**Imports and Setup**

In [1]:
# model and data are defined by this script
# This is a classification model
%run training_script.py
import credoai.lens as cl
import credoai.evaluators as evl

/opt/homebrew/Caskroom/miniforge/base/envs/dev/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


## How do I get my model working with Lens?

The first step in using Lens is creating a `Model` in the Lens framework. You will use different subclasses of `Model` depending on your model type. Currently, `ClassificationModel` and `RegressionModel` are defined.

A model needs to be passed a "model_like" object. This is any object that defines the functions needed by the `Model`. For instance, `ClassificationModel` needs to be passed an object with `predict` function. This object can be a sklearn pipeline, a pytorch model, or any other object that conforms to sklearn's `predict` API.

In [2]:
from credoai.artifacts import ClassificationModel

credo_model = ClassificationModel(name='my_model', model_like=model)

<br>

**Using precomputed values**

A common use case you may run into is wanting to assess *pre-computed* predictions. You don't need Lens to perform inference, just use the inferences you've already generated for assessment.

In order to do so you need to coerce your predictions into a "model-like" object. To do so you make use of "Dummy" Models. Below is an example for `ClassificationModel`

In [3]:
from credoai.artifacts import DummyClassifier
# precomputed predictions
predictions = model.predict(X)
probs = model.predict_proba(X)
# light wrapping to create the dummy model
dummy_model = DummyClassifier(predict_output=predictions, predict_proba_output=probs)
# ... which can then be passed to the ClassificationModel
credo_model_from_predictions = ClassificationModel(name='my_model_name', model_like=dummy_model)

## How do I get my datasets working with Lens?

`Data` is the equivalent of `Model` for datasets. They can be passed to Lens as "assessment_data" (which is the validation data to assess the model against) or as "training_data" (which will not be used to evaluate the model, but will be assessed itself).

`Data` is subclassed for particular data types. Currently `TabularData` is supported, which you can see below:

In [4]:
from credoai.artifacts import TabularData
credo_data = TabularData(name='my_dataset_name',
                          X=X_test,
                          y=y_test,
                          sensitive_features=sensitive_features_test
                          )

A number of things happen under the hood when you set up TabularData. For instance, numpy arrays are transformed into dataframes or series, and (optional) sensitive feature intersections are calculated.

## How do I get assessment results from Lens?

Running assessments isn't very helpful if you can't view them! You can get results by calling `lens.get_results()`

All results will be dataframes.

**Note**

If you want to export the assessments to Credo AI's Governance App, check out the [Connecting with Governance App](https://credoai-lens.readthedocs.io/en/latest/notebooks/governance_integration.html) tutorial for directions.


In [5]:
lens = cl.Lens(model=credo_model,
               assessment_data=credo_data)
results = lens \
    .add(evl.Performance(['precision_score', 'recall_score'])) \
    .add(evl.ModelFairness(['precision_score', 'recall_score'])) \
    .run() \
    .get_results()

2022-10-03 12:23:08,785 - lens - INFO - Evaluator Performance added to pipeline. 
2022-10-03 12:23:09,023 - lens - INFO - Evaluator ModelFairness added to pipeline. Dataset used: assessment_data. Sensitive feature: SEX
2022-10-03 12:23:09,024 - lens - INFO - Running evaluation for step: Performance_45ce84
2022-10-03 12:23:09,025 - lens - INFO - Running evaluation for step: ModelFairness_a39f56


In [6]:
lens.print_results()

Evaluator: Performance_45ce84

              type     value
0  precision_score  0.628081
1     recall_score  0.360172


Evaluator: ModelFairness_a39f56

              type     value subtype
0  precision_score  0.016322  parity
1     recall_score  0.027686  parity

      SEX             type     value
0  female  precision_score  0.618687
1    male  precision_score  0.635009
2  female     recall_score  0.344585
3    male     recall_score  0.372271




## What metrics are available?

Each assessment has different configuration options, as discused above. Some assessments take a set of metrics as their configuration (the FairnessAssessment and PerformanceAssessment).

Many metrics are supported out-of-the-box. These metrics can be referenced by string.

In [7]:
# all out-of-the-box supported metrics can be accessed by calling list_metrics
from credoai.modules import list_metrics
metrics = list_metrics()

BINARY_CLASSIFICATION
	accuracy_score, average_precision_score,
	balanced_accuracy_score, f1_score, fallout_rate,
	false_discovery_rate, false_negative_rate,
	false_omission_rate, false_positive_rate, fdr,
	fnr, fpr, hit_rate,
	matthews_correlation_coefficient, miss_rate,
	overprediction, precision, precision_score,
	recall, recall_score, roc_auc_score,
	selection_rate, sensitivity, specificity, tnr,
	tpr, true_negative_rate, true_positive_rate,
	underprediction

FAIRNESS
	demographic_parity,
	demographic_parity_difference,
	demographic_parity_ratio, disparate_impact,
	equal_opportunity, equal_opportunity_difference,
	equalized_odds, equalized_odds_difference,
	statistical_parity

REGRESSION
	MAE, MSD, MSE, RMSE, d2_tweedie_score,
	explained_variance_score, max_error,
	mean_absolute_error,
	mean_absolute_percentage_error,
	mean_gamma_deviance, mean_pinball_loss,
	mean_poisson_deviance, mean_squared_deviation,
	mean_squared_error, mean_squared_log_error,
	median_absolute_error, r2, r2_s

Under the hood each metric is wrapped in a `Metric` class. `Metrics` are lightweight wrapper classes that defines a few characteristics of the custom function needed by Lens.

This class defines a canonical name for Lens, synonyms, a metric category, the function, and whether the metric takes probabilities or categorical predictions. The metric category defines the expected function signature, as described in `Metric`'s documentation

For instance, below is the false positive rate metric.

In [8]:
from credoai.modules import BINARY_CLASSIFICATION_METRICS
BINARY_CLASSIFICATION_METRICS['false_positive_rate']

Metric(name='false_positive_rate', metric_category='BINARY_CLASSIFICATION', fun=<function false_positive_rate at 0x28ebbd820>, takes_prob=False, equivalent_names={'false_positive_rate', 'fpr', 'fallout_rate'})

## How do I use my own custom metrics?

Custom metrics can be created by using the `Metric` class.  

**Example: Confidence Intervals**

We will create custom metrics that reflect the lower and upper 95th percentile confidence bound on the true positive rate.

Confidence intervals are not supported by default. However, they can be derived for some metrics using the `wilson confidence interval`. We will use a convenience function called `confusion_wilson` returns an array: [lower, upper] bound for metrics like true-positive-rate. 

Wrapping the wilson function in a `Metric` allows us to use it in Lens.

In [9]:
from credoai.modules.credoai_metrics import confusion_wilson
from credoai.modules import Metric

# define partial functions for the true positive rate lower bound
def lower_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='true_positive_rate', confidence=0.95)[0]

# and upper bound
def upper_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='true_positive_rate', confidence=0.95)[1]

# wrap the functions in fairness functions
lower_metric = Metric(name = 'lower_bound_tpr', 
                      metric_category = "binary_classification",
                      fun = lower_bound_tpr)

upper_metric = Metric(name = 'upper_bound_tpr', 
                      metric_category = "binary_classification",
                      fun = upper_bound_tpr)

In [10]:
# run Lens as normal with custom metrics
lens = cl.Lens(model=credo_model,
               assessment_data=credo_data)
lens.add(evl.Performance([lower_metric, 'tpr', upper_metric]))
lens.run().print_results()

2022-10-03 12:23:09,289 - lens - INFO - Evaluator Performance added to pipeline. 
2022-10-03 12:23:09,290 - lens - INFO - Running evaluation for step: Performance_1b1bd0
Evaluator: Performance_1b1bd0

              type     value
0  lower_bound_tpr  0.337201
1              tpr  0.360172
2  upper_bound_tpr  0.383802


